In [50]:
from transformers import BertTokenizer
import tensorflow as tf
from keras.utils import pad_sequences
import pandas as pd
from sklearn.model_selection import train_test_split
from matplotlib.colors import LinearSegmentedColormap
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler
from PIL import Image, ImageFont, ImageDraw
import numpy as np
from xsvmlib.xsvmc import xSVMC
from joblib import dump, load
import base64
from io import BytesIO

In [2]:
RANDOM_STATE = 42

In [3]:
df_news = pd.read_csv("./data/df_total.csv")

In [4]:
sentences = df_news.news.values

In [5]:
df_news.Type.unique()

array(['Otra', 'Regulaciones', 'Alianzas', 'Macroeconomia', 'Innovacion',
       'Sostenibilidad', 'Reputacion'], dtype=object)

In [6]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=True)

In [7]:
max_len = 4500

In [8]:
token_ids = []
encoded_inputs = []
embed_vectors = []
token_to_text = []
for sentence in sentences:
  encoded_input = tokenizer(sentence, return_tensors='tf')
  tokens = encoded_input.input_ids[0]
  
  token_ids.append(tokens)
  encoded_inputs.append(encoded_input)

ready_tokens = pad_sequences(token_ids, padding="post", maxlen=max_len)

for i in range(len(ready_tokens)):
  text = tokenizer.convert_ids_to_tokens(ready_tokens[i])
  token_to_text.append(text) 

Token indices sequence length is longer than the specified maximum sequence length for this model (514 > 512). Running this sequence through the model will result in indexing errors


In [9]:
scaler = MinMaxScaler()
scaler.fit(ready_tokens)

MinMaxScaler()

In [10]:
X = scaler.transform(ready_tokens)
y = df_news["Type"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=RANDOM_STATE)

In [11]:
param_grid = {
  'C': [1, 0.1, 10, 100], 
  'kernel': ["poly", "rbf"],
  'gamma': [0.01, 0.1, 0.5, 0.7, 1],
  'degree': [3, 4, 5],
  'coef0': [0.1, 1, 10, 100],
}

In [12]:
# grid = GridSearchCV(xSVMC(), param_grid, cv=3, verbose=2)
# grid.fit(X_train,y_train)

In [13]:
# grid.best_estimator_

In [14]:
#Best parameter
kernel = "poly"
gamma = 0.1
degree = 3
C =  0.1
k = 3
coef0 = 0.1
clf = xSVMC(kernel=kernel, degree=degree, C=C, gamma=gamma, k=k, coef0=coef0, random_state=RANDOM_STATE)
clf.fit(X_train, y_train)

xSVMC(C=0.1, coef0=0.1, gamma=0.1, k=3, kernel='poly', random_state=42)

In [15]:
SVs = clf.support_vectors_

In [16]:
confusion_matrix(y_test, clf.predict(X_test))

array([[40,  0, 34,  3,  2,  0,  2],
       [ 2, 21, 26,  0,  0,  0,  3],
       [15,  4, 79,  1,  4,  0,  5],
       [23,  0, 15,  0,  2,  0,  0],
       [18,  1, 12,  0,  3,  0,  1],
       [ 2,  0,  6,  1,  1,  0,  0],
       [ 2,  3, 24,  0,  0,  0, 11]], dtype=int64)

In [27]:
print(classification_report(y_test, clf.predict(X_test)))

                precision    recall  f1-score   support

      Alianzas       0.39      0.49      0.44        81
    Innovacion       0.72      0.40      0.52        52
 Macroeconomia       0.40      0.73      0.52       108
          Otra       0.00      0.00      0.00        40
  Regulaciones       0.25      0.09      0.13        35
    Reputacion       0.00      0.00      0.00        10
Sostenibilidad       0.50      0.28      0.35        40

      accuracy                           0.42       366
     macro avg       0.32      0.28      0.28       366
  weighted avg       0.39      0.42      0.37       366



d:\Github Projects\back_news_classifier\.env\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\Github Projects\back_news_classifier\.env\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\Github Projects\back_news_classifier\.env\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

In [18]:
dump(clf, "./xsvmc.joblib")

['./xsvmc.joblib']

In [19]:
colormap = LinearSegmentedColormap.from_list('custom', 
                                       [(0, '#162cd9'),
                                        (1,   '#f2271f')], N=256)

In [20]:
def draw_text(draw_obj, text, pos_x, pos_y, prob, font):
  color = colormap(prob)[:3]
  draw_obj.text((pos_x, pos_y), text, fill=(int(color[0] * 255), int(color[1] * 255), int(color[2] * 255), 255), font=font)

In [21]:
def create_img(l_text, prob, y_size):
  offset_y = 10
  offset_x = 10
  img_x = 800
  img_y = y_size

  img = Image.new(mode="RGBA", size=(img_x, img_y), color = (255, 255, 255))
  txt = Image.new('RGBA', img.size, (255,255,255,0))
  draw = ImageDraw.Draw(txt)
  font = ImageFont.truetype("models/xsvmc_model/fonts/SpaceMono-Bold.ttf", 16)

  last_pos_x = 0
  combined = 0
  pos_y = 0
  for i in range(len(l_text)):
    palabra = l_text[i]
    if palabra.startswith("##"):
      palabra = palabra[2:]
      pos_x = (len(palabra)) * 10
    elif palabra not in [".", ","]:
      palabra = " " + palabra
      pos_x = (len(palabra)) * 10
      if last_pos_x + pos_x + offset_x >= img_x - 70:
        pos_y += 18
        last_pos_x = 0
    else:
      pos_x = (len(palabra)) * 10

    draw_text(draw, palabra, last_pos_x + offset_x, pos_y + offset_y, prob[i], font)
    last_pos_x += pos_x

  combined = Image.alpha_composite(img, txt)
  buffered = BytesIO()
  combined.save(buffered, format="PNG")
  img_str = base64.b64encode(buffered.getvalue())
  return img_str

In [22]:
def render_text(text, misv):
  caracteres = " ".join(text)
  y_size = int(len(caracteres) / 60 * 18)
  values = misv
  max_value = max(values)
  prob = (values / max_value)
  return create_img(text, prob, y_size)

In [23]:
def preprocess_text(text):
  encoded_input = tokenizer(text, return_tensors='tf')
  tokens = encoded_input.input_ids[0]
  ready_tokens = pad_sequences([tokens], padding="post", maxlen=max_len)[0]
  text_tokens = tokenizer.convert_ids_to_tokens(ready_tokens)
  return ready_tokens, text_tokens, len(tokens)

In [24]:
def contextualized_prediction(text):
  tokens, text_tokens, original_len = preprocess_text(text)
  clean_text = text_tokens[1:original_len-1]
  topK = clf.predict_with_context(tokens)
  response = []
  for i in range(len(topK)):
    pred = topK[i]
    mu_misv = SVs[pred.eval.mu_hat.misv_idx][1:original_len-1]
    nu_misv = SVs[pred.eval.nu_hat.misv_idx][1:original_len-1]
    b64_pro = render_text(clean_text, mu_misv)
    b64_contra = render_text(clean_text, nu_misv)
    response.append({
      'clase': pred.class_name,
      'favor': b64_pro,
      'favor_value': pred.eval.mu_hat.level,
      'contra': b64_contra,
      'contra_value': pred.eval.nu_hat.level
    })
  return response